<a href="https://colab.research.google.com/github/emi-emi671/EEG-Anonymization/blob/main/Patient_level_anonymization_with_cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import joblib
import os

CACHE_FILE = "/content/drive/MyDrive/Masters/google-cloab/EEG_CACHE/patient_features_v1.joblib"

if os.path.exists(CACHE_FILE):
    print("✅ Loading cached EEG features...")
    cache = joblib.load(CACHE_FILE)

    patient_vectors_raw = cache["patient_vectors_raw"]
    patient_vectors_norm = cache["patient_vectors_norm"]
    feature_names = cache["feature_names"]
    meta = cache["meta"]
    norm_params = cache["norm_params"]

else:
    print("⚙️ Running EEG preprocessing (this may take time)...")

    patient_vectors_raw, patient_vectors_norm, feature_names, meta, norm_params = (
        build_patient_feature_vectors(
            folder="/content/EEG_DATA/",
            mat_glob="*.mat",
            patient_regex=r"^(Patient\d+)",
            default_fs=256.0,
            band=(0.5, 60.0),
            notch=40.0,
            win_sec=2.0,
            overlap=0.5,
            agg_method="mean",
            log1p_bp=True,
            # normalize="zscore",
        )
    )

    joblib.dump(
        {
            "patient_vectors_raw": patient_vectors_raw,
            "patient_vectors_norm": patient_vectors_norm,
            "feature_names": feature_names,
            "meta": meta,
            "norm_params": norm_params,
        },
        CACHE_FILE,
        compress=3,
    )

✅ Loading cached EEG features...


**EEG pipeline**

In [ ]:
import os
import re
import glob
import numpy as np
import h5py
import scipy.io as sio
from scipy.signal import butter, sosfiltfilt, iirnotch, filtfilt, welch
from typing import Dict, List, Tuple, Optional, Literal


# ============================================================
# 1) Patient ID extraction + file grouping
# ============================================================

_PATIENT_REGEX_DEFAULT = r"^(Patient\d+)"

def extract_patient_id(filename: str, pattern: str = _PATIENT_REGEX_DEFAULT) -> str:
    base = os.path.basename(filename)
    m = re.match(pattern, base)
    if not m:
        raise ValueError(
            f"Could not extract patient id from filename: {base}\n"
            f"Regex used: {pattern}\n"
            f"Update 'pattern' to match your naming scheme."
        )
    return m.group(1)

def group_files_by_patient(folder: str, mat_glob: str = "*.mat",
                           patient_regex: str = _PATIENT_REGEX_DEFAULT) -> Dict[str, List[str]]:
    files = sorted(glob.glob(os.path.join(folder, mat_glob)))
    if not files:
        raise FileNotFoundError(f"No files found in {folder} matching {mat_glob}")

    groups: Dict[str, List[str]] = {}
    for fp in files:
        pid = extract_patient_id(fp, pattern=patient_regex)
        groups.setdefault(pid, []).append(fp)

    for pid in groups:
        groups[pid] = sorted(groups[pid])

    return groups


# ============================================================
# 2) MATLAB/HDF5 loaders
# ============================================================

def _numeric_datasets(h5file):
    items = []
    def visit(name, obj):
        if isinstance(obj, h5py.Dataset) and np.issubdtype(obj.dtype, np.number):
            items.append((name, obj))
    h5file.visititems(visit)
    return items

def _read_scalar_ref(f: h5py.File, ref):
    try:
        v = np.asarray(f[ref])[()]
        v = np.asarray(v).squeeze()
        if v.size == 1 and np.isfinite(v).all():
            return float(v)
    except Exception:
        return None
    return None

def _detect_fs(f: h5py.File, n_ch: int, default_fs: float) -> float:
    if "#refs#/e/samplingRate" in f:
        ds = f["#refs#/e/samplingRate"]
        arr = np.asarray(ds[()]).squeeze()
        if arr.ndim == 1 and arr.size == n_ch and np.allclose(arr, arr[0]):
            val = float(arr[0])
            if 50.0 <= val <= 5000.0:
                return val

    if "#refs#/t6/dSamplingRate" in f:
        ds = f["#refs#/t6/dSamplingRate"]
        try:
            refs = np.asarray(ds[()]).squeeze().reshape(-1)
            vals = []
            for r in refs:
                v = _read_scalar_ref(f, r)
                if v is not None:
                    vals.append(v)
            vals = [v for v in vals if 50.0 <= v <= 5000.0]
            if vals:
                v0 = float(vals[0])
                if all(abs(v - v0) < 1e-6 for v in vals):
                    return v0
        except Exception:
            pass

    return float(default_fs)

def load_eeg_and_fs(mat_path: str, default_fs: float = 256.0, mat_key_fallback: str = "val"):
    try:
        with h5py.File(mat_path, "r") as f:
            best = None
            for path, ds in _numeric_datasets(f):
                if ds.ndim != 2:
                    continue
                r, c = ds.shape
                ch, smp = min(r, c), max(r, c)
                if not (4 <= ch <= 512 and smp / ch >= 10):
                    continue
                score = ds.size + (0.7 * ds.size if np.issubdtype(ds.dtype, np.floating) else 0)
                if best is None or score > best[0]:
                    best = (score, path)

            if best is None:
                raise RuntimeError(f"EEG dataset not found in {mat_path}")

            eeg_path = best[1]
            eeg = np.asarray(f[eeg_path][()])
            if eeg.shape[0] > eeg.shape[1]:
                eeg = eeg.T
            eeg = eeg.astype(np.float32, copy=False)

            fs = _detect_fs(f, n_ch=eeg.shape[0], default_fs=default_fs)

        return eeg, fs, eeg_path

    except OSError:
        d = sio.loadmat(mat_path)
        if mat_key_fallback not in d:
            raise RuntimeError(f"'{mat_key_fallback}' not found in {mat_path}. Keys: {list(d.keys())}")

        eeg = d[mat_key_fallback]
        if eeg.shape[0] > eeg.shape[1]:
            eeg = eeg.T
        eeg = eeg.astype(np.float32, copy=False)
        fs = float(default_fs)
        return eeg, fs, mat_key_fallback


# ============================================================
# 3) Preprocessing + Features
# ============================================================

def preprocess(eeg: np.ndarray, fs: float,
               band: Tuple[float, float] = (0.5, 70.0),
               notch: Optional[float] = 50.0,
               reref: bool = True) -> np.ndarray:
    x = eeg.astype(np.float64, copy=False)
    nyq = 0.5 * fs
    low, high = band
    if not (0 < low < high < nyq):
        raise ValueError(f"Bad band {band} for fs={fs} (Nyq={nyq}).")

    sos = butter(4, [low / nyq, high / nyq], btype="band", output="sos")
    x = sosfiltfilt(sos, x, axis=1)

    if notch is not None:
        if not (0 < notch < nyq):
            raise ValueError(f"Bad notch {notch} for fs={fs} (Nyq={nyq}).")
        b, a = iirnotch(notch / nyq, Q=30.0)
        x = filtfilt(b, a, x, axis=1)

    if reref:
        x = x - x.mean(axis=0, keepdims=True)

    return x.astype(np.float32, copy=False)

def extract_window_features(eeg_prep: np.ndarray, fs: float,
                            win_sec: float = 2.0, overlap: float = 0.5):
    bands = {
        "delta": (0.5, 4), "theta": (4, 8), "alpha": (8, 13),
        "beta": (13, 30), "gamma": (30, 45), "high_gamma": (45, 70)
    }
    n_ch, n_samp = eeg_prep.shape
    win = int(round(win_sec * fs))
    step = int(round(win * (1 - overlap)))
    if win <= 1 or step <= 0:
        raise ValueError("Bad window parameters; check win_sec/overlap/fs.")
    nper = min(256, win)

    def bandpower(f, p, lo, hi):
        m = (f >= lo) & (f < hi)
        return float(np.trapezoid(p[m], f[m])) if np.any(m) else 0.0

    def sef95(f, p):
        c = np.cumsum(np.maximum(p, 0.0))
        if c[-1] <= 0:
            return 0.0
        return float(f[np.searchsorted(c, 0.95 * c[-1])])

    feat_names = []
    for ch in range(n_ch):
        feat_names += [f"ch{ch}_ll", f"ch{ch}_rms", f"ch{ch}_var", f"ch{ch}_zcr"]
        feat_names += [f"ch{ch}_{b}_bp" for b in bands]
        feat_names += [f"ch{ch}_sef95"]

    X = []
    for s in range(0, n_samp - win + 1, step):
        seg = eeg_prep[:, s:s + win]
        row = []
        for ch in range(n_ch):
            x = seg[ch].astype(np.float64, copy=False)
            row += [
                float(np.sum(np.abs(np.diff(x)))) if x.size > 1 else 0.0,
                float(np.sqrt(np.mean(x * x))),
                float(np.var(x)),
                float(np.mean(x[:-1] * x[1:] < 0)) if x.size > 1 else 0.0,
            ]
            f, p = welch(x, fs=fs, nperseg=nper, noverlap=nper // 2)
            for lo, hi in bands.values():
                row.append(bandpower(f, p, lo, hi))
            row.append(sef95(f, p))
        X.append(row)

    return np.asarray(X, dtype=np.float32), feat_names


# ============================================================
# 4) Patient aggregation + optional log1p
# ============================================================

def log1p_bandpower_vector(v: np.ndarray, feature_names: List[str]) -> np.ndarray:
    v = v.copy()
    bp_idx = [i for i, n in enumerate(feature_names) if "_bp" in n]
    if bp_idx:
        v[bp_idx] = np.log1p(np.maximum(v[bp_idx], 0.0))
    return v

def aggregate_patient_windows(X_all_windows: np.ndarray, method: str = "mean") -> np.ndarray:
    if X_all_windows.ndim != 2 or X_all_windows.shape[0] == 0:
        raise ValueError("Need non-empty window matrix (n_windows, n_features).")

    if method == "mean":
        return X_all_windows.mean(axis=0)
    if method == "median":
        return np.median(X_all_windows, axis=0)
    if method == "mean_std":
        mu = X_all_windows.mean(axis=0)
        sd = X_all_windows.std(axis=0, ddof=0)
        return np.concatenate([mu, sd], axis=0)
    raise ValueError("method must be one of: 'mean', 'median', 'mean_std'")


# ============================================================
# 5) NEW: Normalizer across patients (zscore / robust / minmax)
# ============================================================

NormMethod = Literal["none", "zscore", "robust", "minmax"]

def fit_normalizer(X: np.ndarray, method: NormMethod):
    X = np.asarray(X, dtype=np.float64)

    if method == "none":
        return {"method": "none"}

    if method == "zscore":
        mean = X.mean(axis=0)
        std = X.std(axis=0, ddof=0)
        std = np.where(std > 0, std, 1.0)
        return {"method": "zscore", "mean": mean, "std": std}

    if method == "robust":
        med = np.median(X, axis=0)
        q1 = np.quantile(X, 0.25, axis=0)
        q3 = np.quantile(X, 0.75, axis=0)
        iqr = q3 - q1
        iqr = np.where(iqr > 0, iqr, 1.0)
        return {"method": "robust", "median": med, "iqr": iqr}

    if method == "minmax":
        mn = X.min(axis=0)
        mx = X.max(axis=0)
        rng = mx - mn
        rng = np.where(rng > 0, rng, 1.0)
        return {"method": "minmax", "min": mn, "range": rng}

    raise ValueError("Unknown method")

def transform_with_normalizer(X: np.ndarray, params: dict) -> np.ndarray:
    X = np.asarray(X, dtype=np.float64)
    m = params["method"]

    if m == "none":
        return X.astype(np.float32)

    if m == "zscore":
        return ((X - params["mean"]) / params["std"]).astype(np.float32)

    if m == "robust":
        return ((X - params["median"]) / params["iqr"]).astype(np.float32)

    if m == "minmax":
        return ((X - params["min"]) / params["range"]).astype(np.float32)

    raise ValueError("Bad normalizer params")


# ============================================================
# 6) Main: build patient vectors + (optional) normalize
# ============================================================

def build_patient_feature_vectors(
    folder: str,
    *,
    mat_glob: str = "*.mat",
    patient_regex: str = _PATIENT_REGEX_DEFAULT,
    default_fs: float = 256.0,
    mat_key_fallback: str = "val",
    band: Tuple[float, float] = (0.5, 60.0),
    notch: Optional[float] = 50.0,
    reref: bool = True,
    win_sec: float = 2.0,
    overlap: float = 0.5,
    agg_method: str = "mean",
    log1p_bp: bool = True,
    normalize: NormMethod = "none",   # <-- NEW
) -> Tuple[Dict[str, np.ndarray], Dict[str, np.ndarray], List[str], Dict[str, dict], dict]:
    """
    Returns:
      patient_vectors_raw : dict patient_id -> raw aggregated vector
      patient_vectors_norm: dict patient_id -> normalized vector (if normalize != 'none')
      feature_names       : list[str]
      metadata            : dict
      norm_params         : dict (normalizer parameters)
    """
    groups = group_files_by_patient(folder, mat_glob=mat_glob, patient_regex=patient_regex)

    patient_vectors_raw: Dict[str, np.ndarray] = {}
    metadata: Dict[str, dict] = {}
    feat_names_ref: Optional[List[str]] = None

    for pid, file_list in groups.items():
        windows_all = []
        per_file_info = []

        for fp in file_list:
            eeg, fs, eeg_path = load_eeg_and_fs(fp, default_fs=default_fs, mat_key_fallback=mat_key_fallback)
            eeg_prep = preprocess(eeg, fs, band=band, notch=notch, reref=reref)
            Xw, feat_names = extract_window_features(eeg_prep, fs, win_sec=win_sec, overlap=overlap)

            if feat_names_ref is None:
                feat_names_ref = feat_names
            elif feat_names != feat_names_ref:
                raise ValueError(f"Feature mismatch detected. File: {fp}")

            windows_all.append(Xw)
            per_file_info.append({
                "file": os.path.basename(fp),
                "path": fp,
                "fs": float(fs),
                "eeg_path": eeg_path,
                "n_windows": int(Xw.shape[0]),
                "n_channels": int(eeg.shape[0]),
                "n_samples": int(eeg.shape[1]),
            })

        if not windows_all or sum(w.shape[0] for w in windows_all) == 0:
          raise ValueError(f"No valid windows for patient {pid}")

        X_patient_windows = np.vstack(windows_all)
        v = aggregate_patient_windows(X_patient_windows, method=agg_method)

        # feature name expansion for mean_std
        feature_names = feat_names_ref
        if agg_method == "mean_std":
            feature_names = [f"{n}_mean" for n in feat_names_ref] + [f"{n}_std" for n in feat_names_ref]

        if log1p_bp:
            v = log1p_bandpower_vector(v, feature_names)

        patient_vectors_raw[pid] = v.astype(np.float32, copy=False)
        metadata[pid] = {
            "patient_id": pid,
            "n_files": len(file_list),
            "files": per_file_info,
            "total_windows": int(X_patient_windows.shape[0]),
            "agg_method": agg_method,
        }

    if feat_names_ref is None:
        raise RuntimeError("No features extracted.")

    final_feature_names = feat_names_ref
    if agg_method == "mean_std":
        final_feature_names = [f"{n}_mean" for n in feat_names_ref] + [f"{n}_std" for n in feat_names_ref]

    # ----- normalization across patients (optional) -----
    pids_sorted = sorted(patient_vectors_raw.keys())
    X_raw = np.stack([patient_vectors_raw[k] for k in pids_sorted])

    norm_params = fit_normalizer(X_raw, method=normalize)

    if normalize == "none":
        X_norm = X_raw.astype(np.float32)
    else:
        X_norm = transform_with_normalizer(X_raw, norm_params)

    patient_vectors_norm: Dict[str, np.ndarray] = {}
    for i, pid in enumerate(pids_sorted):
        patient_vectors_norm[pid] = X_norm[i].astype(np.float32, copy=False)

    return patient_vectors_raw, patient_vectors_norm, final_feature_names, metadata, norm_params

**Test** EEG pipeline from cache or normal

In [4]:
print("Patients:", len(patient_vectors_raw))
first_pid = sorted(patient_vectors_raw.keys())[0]
print("Example patient:", first_pid, "raw shape:", patient_vectors_raw[first_pid].shape)
print("Example patient:", first_pid, "norm shape:", patient_vectors_norm[first_pid].shape)
print("Feature count:", len(feature_names))
print("Files for", first_pid, "=", meta[first_pid]["n_files"], "Total windows:", meta[first_pid]["total_windows"])



Patients: 9
Example patient: Patient1 raw shape: (363,)
Example patient: Patient1 norm shape: (363,)
Feature count: 363
Files for Patient1 = 7 Total windows: 12619


In [3]:
pip install diffprivlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.9/176.9 kB 3.9 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
from diffprivlib.mechanisms import Gaussian

# ============================================================
# COMPLETE Gaussian DP IMPLEMENTATION (no external bounds needed)
# - Auto-computes per-feature bounds from available vectors (robust MAD bounds)
# - Clips per feature
# - Splits (epsilon, delta) across features (basic composition)
# - Adds per-feature Gaussian noise using per-feature sensitivity = (hi - lo)
# ============================================================

def _auto_bounds_mad(X: np.ndarray, k: float = 6.0, min_range: float = 1.0) -> tuple:
    """
    Robust bounds using median +/- k * MAD (scaled).
    Works even for 1 patient (falls back to value +/- min_range/2 if MAD=0).
    """
    X = np.asarray(X, dtype=np.float64)
    med = np.median(X, axis=0)
    mad = np.median(np.abs(X - med), axis=0)  # median absolute deviation
    mad_sigma = 1.4826 * mad  # approx std if normal

    lo = med - k * mad_sigma
    hi = med + k * mad_sigma

    # if range collapses, widen to at least min_range around median
    rng = hi - lo
    bad = rng < min_range
    if np.any(bad):
        lo[bad] = med[bad] - 0.5 * min_range
        hi[bad] = med[bad] + 0.5 * min_range

    return lo, hi


def gaussian_dp_patient_vectors_auto_bounds(
    patient_vectors: dict,          # {pid: 1D np.array}
    feature_names: list,            # list of names, len == n_features
    *,
    epsilon: float = 0.9,
    delta: float = 1e-5,
    clip_k: float = 6.0,            # bounds = median +/- clip_k*MAD_sigma
    min_range: float = 1.0,         # minimum (hi-lo) per feature to avoid zero sensitivity
    seed: int = 0,
) -> tuple:
    """
    Returns:
      patient_vectors_dp : dict {pid: dp_vector}
      report             : dict
      df_example          : DataFrame (first patient dp vector)
    """
    if not patient_vectors:
        raise ValueError("patient_vectors is empty")

    pids = sorted(patient_vectors.keys())
    X = np.stack([np.asarray(patient_vectors[pid], dtype=np.float64) for pid in pids])
    if X.ndim != 2:
        raise ValueError("patient_vectors must stack into 2D array (n_patients, n_features)")

    n_patients, d = X.shape
    if len(feature_names) != d:
        raise ValueError(f"feature_names length ({len(feature_names)}) != n_features ({d})")

    # 1) Auto bounds + clip
    lo, hi = _auto_bounds_mad(X, k=clip_k, min_range=min_range)
    X_clipped = np.clip(X, lo, hi)

    # 2) Basic composition across features
    eps_j = float(epsilon) / float(d)
    del_j = float(delta) / float(d)

    # 3) Per-feature Gaussian
    X_dp = np.empty_like(X_clipped, dtype=np.float64)
    for j in range(d):
        sens_j = float(hi[j] - lo[j])  # per-feature sensitivity after clipping
        mech = Gaussian(epsilon=eps_j, delta=del_j, sensitivity=sens_j)
        for i in range(n_patients):
            X_dp[i, j] = mech.randomise(float(X_clipped[i, j]))

    patient_vectors_dp = {pid: X_dp[i].astype(np.float32) for i, pid in enumerate(pids)}

    # 4) Quick diagnostics (vs clipped baseline)
    diff = X_dp - X_clipped
    mse = float(np.mean(diff ** 2))
    rmse = float(np.sqrt(mse))
    mae = float(np.mean(np.abs(diff)))
    max_abs = float(np.max(np.abs(diff)))

    report = {
        "patients": int(n_patients),
        "features": int(d),
        "epsilon_total": float(epsilon),
        "delta_total": float(delta),
        "epsilon_per_feature": float(eps_j),
        "delta_per_feature": float(del_j),
        "clip_k": float(clip_k),
        "min_range": float(min_range),
        "rmse_vs_clipped": rmse,
        "mae_vs_clipped": mae,
        "max_abs_change": max_abs,
        "avg_sensitivity": float(np.mean(hi - lo)),
        "min_sensitivity": float(np.min(hi - lo)),
        "max_sensitivity": float(np.max(hi - lo)),
    }

    first_pid = pids[0]
    df_example = pd.DataFrame([patient_vectors_dp[first_pid]], columns=feature_names)

    return patient_vectors_dp, report, df_example


# ============================
# RUN (use raw or normalized vectors)
# ============================
# For your case:
# - If you have ONLY 1 patient, using patient_vectors_raw is usually better than zscore across patients.
# - If you have many patients, you can use patient_vectors_norm.

patient_vectors_dp, report, df_dp = gaussian_dp_patient_vectors_auto_bounds(
    patient_vectors=patient_vectors_raw,   # or patient_vectors_norm
    feature_names=feature_names,
    epsilon=0.9,
    delta=1e-5,
    clip_k=6.0,
    min_range=1.0,
    seed=0,
)

#print(report)

first_pid = sorted(patient_vectors_raw.keys())[0]
print("RAW first 5:", patient_vectors_raw[first_pid][:5])
print("DP  first 5:", patient_vectors_dp[first_pid][:5])
print(df_dp.iloc[:, :5])

RAW first 5: [1.1923248e+04 4.8982956e+01 3.6722339e+03 1.7786542e-01 6.0495629e+00]
DP  first 5: [-3.0296155e+06 -2.4677898e+04  1.7494292e+07 -1.4906648e+03
  5.2986680e+03]
      ch0_ll       ch0_rms     ch0_var      ch0_zcr  ch0_delta_bp
0 -3029615.5 -24677.898438  17494292.0 -1490.664795   5298.667969


Final Gaussian test

In [7]:
import numpy as np
import pandas as pd

# Inputs you already have from the last code:
# patient_vectors     : dict patient_id -> original vector (1D)
# patient_vectors_dp  : dict patient_id -> DP vector (1D)
# feature_names       : list[str] (optional, for display)

# ---------- Compute utility metrics ----------
rows = []
for pid in sorted(patient_vectors_raw.keys()):
    x = np.asarray(patient_vectors_raw[pid], dtype=np.float64).reshape(-1)
    xdp = np.asarray(patient_vectors_dp[pid], dtype=np.float64).reshape(-1)

    mse = float(np.mean((x - xdp) ** 2))
    signal_power = float(np.mean(x ** 2))
    snr = float(10 * np.log10(signal_power / max(mse, 1e-12)))

    mae = float(np.mean(np.abs(x - xdp)))
    max_abs = float(np.max(np.abs(x - xdp)))

    rows.append({
        "patient_id": pid,
        "mse": mse,
        "snr_db": snr,
        "mae": mae,
        "max_abs_change": max_abs,
    })

df_metrics = pd.DataFrame(rows)

print("Per-patient utility metrics (first 10):")
print(df_metrics.head(10))

print("\nOverall summary:")
print(df_metrics[["mse", "snr_db", "mae", "max_abs_change"]].describe())

# ---------- Optional: show one patient vector comparison ----------
example_pid = df_metrics.sort_values("mse", ascending=False).iloc[0]["patient_id"]  # worst MSE patient
print("\nExample patient (worst MSE):", example_pid)

if "feature_names" in globals() and feature_names is not None:
    df_compare = pd.DataFrame({
        "feature": feature_names,
        "original": np.asarray(patient_vectors_raw[example_pid], dtype=np.float64),
        "dp": np.asarray(patient_vectors_dp[example_pid], dtype=np.float64),
    })
    df_compare["abs_diff"] = np.abs(df_compare["original"] - df_compare["dp"])
    print(df_compare.sort_values("abs_diff", ascending=False).head(15))
else:
    # fallback without feature names
    x = np.asarray(patient_vectors_raw[example_pid], dtype=np.float64)
    xdp = np.asarray(patient_vectors_dp[example_pid], dtype=np.float64)
    diffs = np.abs(x - xdp)
    top = np.argsort(diffs)[-15:][::-1]
    print("Top-15 changed indices:", top)
    print("Abs diffs:", diffs[top])

Per-patient utility metrics (first 10):
  patient_id           mse     snr_db           mae  max_abs_change
0   Patient1  1.241247e+15 -69.554816  6.755596e+06    3.391633e+08
1  Patient10  1.842728e+15 -79.020357  8.295060e+06    4.598627e+08
2  Patient11  2.417865e+15 -82.495008  9.125359e+06    5.612969e+08
3  Patient12  1.813328e+15 -82.048387  7.874787e+06    4.580509e+08
4  Patient13  5.357086e+15 -89.127100  1.043757e+07    1.128990e+09
5  Patient17  6.127747e+15 -81.063009  9.708198e+06    1.305286e+09
6   Patient7  7.970483e+15 -85.845577  1.219768e+07    1.517381e+09
7   Patient8  9.829199e+15 -90.412414  1.061657e+07    1.801629e+09
8   Patient9  3.120971e+15 -78.744960  1.136452e+07    5.391990e+08

Overall summary:
                mse     snr_db           mae  max_abs_change
count  9.000000e+00   9.000000  9.000000e+00    9.000000e+00
mean   4.413406e+15 -82.034625  9.597259e+06    9.012065e+08
std    3.059760e+15   6.243230  1.753681e+06    5.430525e+08
min    1.241247e+1

In [9]:
# ============================================================
# PSD + BANDPOWER UTILITY CHECK (ORIGINAL vs DP)
# Works on patient_vectors_raw and patient_vectors_dp
# ============================================================

import numpy as np

# -------- helper: map feature indices ----------
def get_bandpower_indices(feature_names):
    bands = ["delta", "theta", "alpha", "beta", "gamma", "high_gamma"]
    band_idx = {b: [] for b in bands}
    for i, name in enumerate(feature_names):
        for b in bands:
            if f"_{b}_bp" in name:
                band_idx[b].append(i)
    return band_idx


# -------- extract matrices ----------
pids = sorted(patient_vectors_raw.keys())

X_orig = np.stack([patient_vectors_raw[pid] for pid in pids])
X_dp   = np.stack([patient_vectors_dp[pid] for pid in pids])

# -------- bandpower indices ----------
band_idx = get_bandpower_indices(feature_names)

# -------- bandpower error ----------
print("\n=== BANDPOWER ERRORS ===")

for band, idx in band_idx.items():
    if len(idx) == 0:
        continue

    orig = X_orig[:, idx]
    dp   = X_dp[:, idx]

    mse  = np.mean((dp - orig) ** 2)
    rmse = np.sqrt(mse)
    mae  = np.mean(np.abs(dp - orig))

    # correlation (flatten)
    if orig.size > 1:
        corr = np.corrcoef(orig.flatten(), dp.flatten())[0, 1]
    else:
        corr = np.nan

    print(f"{band.upper():>10} | RMSE={rmse:.4f}  MAE={mae:.4f}  Corr={corr:.4f}")


# -------- relative error ----------
print("\n=== RELATIVE ERROR (%) ===")

for band, idx in band_idx.items():
    if len(idx) == 0:
        continue

    orig = X_orig[:, idx]
    dp   = X_dp[:, idx]

    rel = np.abs(dp - orig) / (np.abs(orig) + 1e-8)
    rel_mean = np.mean(rel) * 100

    print(f"{band.upper():>10} | RelError={rel_mean:.2f}%")


# -------- global summary ----------
print("\n=== GLOBAL SUMMARY ===")

mse  = np.mean((X_dp - X_orig) ** 2)
rmse = np.sqrt(mse)
mae  = np.mean(np.abs(X_dp - X_orig))

if X_orig.size > 1:
    corr = np.corrcoef(X_orig.flatten(), X_dp.flatten())[0, 1]
else:
    corr = np.nan

print("RMSE:", rmse)
print("MAE :", mae)
print("Correlation:", corr)


# ============================================================
# INTERPRETATION GUIDE
# ============================================================

# GOOD UTILITY:
# Corr > 0.8
# Relative error < 20%

# MEDIUM:
# Corr 0.5–0.8
# Relative error 20–50%

# BAD:
# Corr < 0.5
# Relative error > 50%



=== BANDPOWER ERRORS ===
     DELTA | RMSE=34997.7188  MAE=26912.0273  Corr=-0.0152
     THETA | RMSE=24140.4512  MAE=16876.2715  Corr=-0.0408
     ALPHA | RMSE=23765.9082  MAE=15161.3066  Corr=0.1241
      BETA | RMSE=48621.0195  MAE=31484.3027  Corr=0.1873
     GAMMA | RMSE=15060.6123  MAE=8975.5996  Corr=0.0295
HIGH_GAMMA | RMSE=15593.5967  MAE=9050.4736  Corr=0.0146

=== RELATIVE ERROR (%) ===
     DELTA | RelError=562076.69%
     THETA | RelError=405277.44%
     ALPHA | RelError=403666.50%
      BETA | RelError=630338.25%
     GAMMA | RelError=269709.00%
HIGH_GAMMA | RelError=274832.88%

=== GLOBAL SUMMARY ===
RMSE: 66433470.0
MAE : 9597259.0
Correlation: 0.07251893532302549


LapLace implementation:::

In [13]:
import numpy as np

# Convert dict to matrix
if isinstance(patient_vectors_norm, dict):
    patient_ids = list(patient_vectors_norm.keys())
    X = np.array([patient_vectors_norm[pid] for pid in patient_ids], dtype=float)
else:
    X = np.asarray(patient_vectors_norm, dtype=float)

print("Shape:", X.shape)


Shape: (9, 363)


In [26]:
import numpy as np
from diffprivlib.mechanisms import Laplace
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def dp_laplace_patient_latent_best(X, epsilon=10.0, k=1, clip_B=1.0, random_state=0):
    X = np.asarray(X, dtype=float)
    n_patients, n_features = X.shape
    k = int(min(k, n_patients - 1, n_features))
    if k < 1:
        raise ValueError("Need at least 2 patients to use PCA with k>=1.")

    # Normalize heavy-tailed EEG features
    X_log = np.log1p(X)
    X_std = StandardScaler().fit_transform(X_log)

    # PCA -> latent
    pca = PCA(n_components=k, random_state=random_state)
    Z = pca.fit_transform(X_std)

    # Clip latent
    Z_clean = np.clip(Z, -clip_B, clip_B)

    # DP Laplace on latent (composition across k)
    eps_per = float(epsilon) / float(k)
    sens = 2.0 * float(clip_B)
    mech = Laplace(epsilon=eps_per, sensitivity=sens)

    Z_dp = np.empty_like(Z_clean)
    for i in range(Z_clean.shape[0]):
        for j in range(Z_clean.shape[1]):
            Z_dp[i, j] = mech.randomise(float(Z_clean[i, j]))

    # Utility metrics
    mse = float(np.mean((Z_clean - Z_dp) ** 2))
    corr = float(np.corrcoef(Z_clean.reshape(-1), Z_dp.reshape(-1))[0, 1])
    signal_power = float(np.mean(Z_clean ** 2) + 1e-12)
    noise_power = float(np.mean((Z_clean - Z_dp) ** 2) + 1e-12)
    snr_db = float(10 * np.log10(signal_power / noise_power))

    print({"epsilon": epsilon, "k": k, "clip_B": clip_B, "mse": mse, "corr": corr, "snr_db": snr_db})

    return Z_clean, Z_dp

# --- RUN (X is your patient-level matrix, shape (9, 363)) ---
Z_clean, Z_dp = dp_laplace_patient_latent_best(X, epsilon=5.0, k=1, clip_B=1.0)



{'epsilon': 5.0, 'k': 1, 'clip_B': 1.0, 'mse': 0.32590453152115395, 'corr': 0.8916855473215305, 'snr_db': 4.869096008688773}


In [1]:
import numpy as np
import pandas as pd

# Inputs you already have from the last code:
# patient_vectors     : dict patient_id -> original vector (1D)
# patient_vectors_dp  : dict patient_id -> DP vector (1D)
# feature_names       : list[str] (optional, for display)

# ---------- Compute utility metrics ----------
rows = []
for pid in sorted(patient_vectors_raw.keys()):
    x = np.asarray(patient_vectors_raw[pid], dtype=np.float64).reshape(-1)
    xdp = np.asarray(patient_vectors_dp[pid], dtype=np.float64).reshape(-1)

    mse = float(np.mean((x - xdp) ** 2))
    signal_power = float(np.mean(x ** 2))
    snr = float(10 * np.log10(signal_power / max(mse, 1e-12)))

    mae = float(np.mean(np.abs(x - xdp)))
    max_abs = float(np.max(np.abs(x - xdp)))

    rows.append({
        "patient_id": pid,
        "mse": mse,
        "snr_db": snr,
        "mae": mae,
        "max_abs_change": max_abs,
    })

df_metrics = pd.DataFrame(rows)

print("Per-patient utility metrics (first 10):")
print(df_metrics.head(10))

print("\nOverall summary:")
print(df_metrics[["mse", "snr_db", "mae", "max_abs_change"]].describe())

# ---------- Optional: show one patient vector comparison ----------
example_pid = df_metrics.sort_values("mse", ascending=False).iloc[0]["patient_id"]  # worst MSE patient
print("\nExample patient (worst MSE):", example_pid)

if "feature_names" in globals() and feature_names is not None:
    df_compare = pd.DataFrame({
        "feature": feature_names,
        "original": np.asarray(patient_vectors_raw[example_pid], dtype=np.float64),
        "dp": np.asarray(patient_vectors_dp[example_pid], dtype=np.float64),
    })
    df_compare["abs_diff"] = np.abs(df_compare["original"] - df_compare["dp"])
    print(df_compare.sort_values("abs_diff", ascending=False).head(15))
else:
    # fallback without feature names
    x = np.asarray(patient_vectors_raw[example_pid], dtype=np.float64)
    xdp = np.asarray(patient_vectors_dp[example_pid], dtype=np.float64)
    diffs = np.abs(x - xdp)
    top = np.argsort(diffs)[-15:][::-1]
    print("Top-15 changed indices:", top)
    print("Abs diffs:", diffs[top])

NameError: name 'patient_vectors_raw' is not defined